Installing Required packages 

In [53]:
!pip install python-docx
!pip install nltk
!pip install keras
!pip install gensim
!pip install tensorflow
!pip install sklearn
!pip install numpy

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' c

Reading Data's From s3 buckets

In [54]:
import pandas as pd
import boto3
bucket = "projectworks"
file_name = "Cancer.csv"
s3 = boto3.client('s3') 
obj = s3.get_object(Bucket= bucket, Key= file_name) 
initial_df = pd.read_csv(obj['Body'])
data=initial_df["Description"]

Reading Text from Data Frame

In [55]:
voca=[]
text=''
temp_text=''
ct=0
l = list(data)
for i in l:
    text = text+""+i
    voca.append(i)   
    

Data cleaning

In [56]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def Data_cleaning(text):
    
    stop_words=set(stopwords.words('english'))

    words=word_tokenize(text)

    punctuated_sentence=[]

    for i in words:
        if i not in string.punctuation:
            punctuated_sentence.append(i)
          
    Preprocessed_data=""
    for i in (punctuated_sentence):
      Preprocessed_data=Preprocessed_data+" "+i
      
   
    return Preprocessed_data
  

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Y Value Assigning

In [57]:
def find_y(voca):
    c=len(voca)
    y=[1 for i in range(0,c)]
    for i in range(0,c) :
        Word=nltk.word_tokenize(voca[i])
        if 'II' in Word:
            y[i]=2
        if 'III' in Word:
            y[i]=3
        if 'IV' in Word:
            y[i]=4
        if 'V' in Word:
            y[i]=5
        if 'VI' in Word:
            y[i]=6    
        
    return y

Word2Vec

In [59]:
import gensim
from gensim import corpora,models,similarities
m=gensim.models.Word2Vec(l,min_count=1)

vocab_size=len(m.wv.vocab)
print(vocab_size)
data=[]
  
cleaned_data=[]
for i in range(0,len(voca)):
  cleaned_data.append(Data_cleaning(voca[i]))

l=[]
for i in range(0,len(cleaned_data)):
  l.append(cleaned_data[1].split(" "))

for i in range(0,len(cleaned_data)):
  t=[]
  for j in range(0,len(l[i])):
    t.append(m[l[i][j]])
  data.append(t)


99


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Padding Data For RNN Model

In [60]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

t = Tokenizer()
t.fit_on_texts(cleaned_data)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(cleaned_data)
print(encoded_docs)

max_length = max([len(i) for i in cleaned_data])
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)


[[46, 27, 32, 19, 44, 47, 49, 72, 124, 96, 3, 5, 6, 101, 122, 7, 27, 32, 161, 7, 61, 137, 133, 51, 69, 34, 26, 16, 2, 20, 10, 30, 31, 21, 2, 14, 17, 36, 9, 23, 24, 28, 18, 2, 1, 9, 29, 22, 33, 11, 4, 74, 59, 108, 138, 15, 236, 3, 66, 109, 37, 57, 35, 40, 5, 6, 90, 70, 51, 69, 74, 162, 52, 479, 15, 370, 3, 66, 82, 37, 57, 35, 40, 51, 69, 13, 4, 259, 59, 152, 3, 66, 125, 126, 82, 37, 57, 35, 40, 67, 95, 15, 237, 145, 15, 238, 10, 239, 15, 480, 337, 51, 69, 61, 106, 103, 8, 25], [46, 27, 32, 19, 44, 47, 49, 74, 65, 54, 62, 60, 131, 41, 154, 43, 3, 5, 6, 112, 10, 80, 55, 7, 13, 4, 1, 42, 56, 34, 26, 16, 2, 20, 10, 30, 31, 21, 2, 14, 17, 36, 9, 23, 24, 28, 18, 2, 1, 9, 29, 22, 33, 1, 75, 7, 8, 58, 3, 93, 91, 13, 4, 74, 65, 54, 162, 119, 111, 12, 38, 73, 5, 6, 188, 117, 141, 37, 83, 35, 40, 41, 53, 43, 195, 7, 167, 73, 196, 1, 87, 7, 63, 276, 100, 277, 278, 11, 4, 1, 71, 12, 52, 3, 5, 11, 4, 1, 8, 25, 50, 13, 4, 74, 65, 54, 162, 119, 111, 12, 38, 73, 5, 6, 188, 117, 141, 37, 83, 35, 40, 41, 

RNN MODEL

In [61]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import LSTM

model=Sequential()
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(LSTM(5, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(6, activation = 'relu'))
#model.add(Flatten())
#model.add(Dense(5, activation=''))


#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1956, 5)           3030      
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 36        
Total params: 3,376
Trainable params: 3,376
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
y=find_y(voca)
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [63]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(padded_docs,dummy_y,random_state = 0)


model.fit(x_train,y_train, epochs=100, verbose=0)
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("loss",loss*100)
print("Accuracy",accuracy*100)

loss 111.90278880736406
Accuracy 83.33333134651184
